In [1]:
# dependencies
import pandas as pd
import numpy as np


In [2]:
# import csvs

# cleaned data

nuke_file = pd.read_csv("data/cleaned_nuke_csv")

countries_file = pd.read_csv("data/countries.csv")


In [3]:
# import to data bases
nuke_df = pd.DataFrame(nuke_file)

countries_df = pd.DataFrame(countries_file)

nuke_df.head()

,Unnamed: 0,Country,Code,Year,Quantity of Nuclear Weapons,Country_Year,GDP (Current LCU)
0,0,China,CHN,1960,0,China 1960,1483.49
1,1,China,CHN,1961,0,China 1961,1089.96
2,2,China,CHN,1962,0,China 1962,1020.73
3,3,China,CHN,1963,0,China 1963,1098.53
4,4,China,CHN,1964,1,China 1964,1268.46


In [4]:
countries_df

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [5]:
# drop country column in countries_df
countries_df = countries_df.drop(columns=["country"])

# countries_df.head()

In [6]:
# renaming columns
countries_df = countries_df.rename(columns={"latitude":"Latitude", "longitude":"Longitude", "name":"Country"})
countries_df.head()

,Latitude,Longitude,Country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


In [7]:
merge_ltlng = pd.merge(nuke_df, countries_df, on='Country')

merge_ltlng.head()

,Unnamed: 0,Country,Code,Year,Quantity of Nuclear Weapons,Country_Year,GDP (Current LCU),Latitude,Longitude
0,0,China,CHN,1960,0,China 1960,1483.49,35.86166,104.195397
1,1,China,CHN,1961,0,China 1961,1089.96,35.86166,104.195397
2,2,China,CHN,1962,0,China 1962,1020.73,35.86166,104.195397
3,3,China,CHN,1963,0,China 1963,1098.53,35.86166,104.195397
4,4,China,CHN,1964,1,China 1964,1268.46,35.86166,104.195397


In [9]:
# drop unnamed index from merge_ltlng

lat_merged_df = merge_ltlng.drop(columns=["Unnamed: 0"])

lat_merged_df.head()

,Country,Code,Year,Quantity of Nuclear Weapons,Country_Year,GDP (Current LCU),Latitude,Longitude
0,China,CHN,1960,0,China 1960,1483.49,35.86166,104.195397
1,China,CHN,1961,0,China 1961,1089.96,35.86166,104.195397
2,China,CHN,1962,0,China 1962,1020.73,35.86166,104.195397
3,China,CHN,1963,0,China 1963,1098.53,35.86166,104.195397
4,China,CHN,1964,1,China 1964,1268.46,35.86166,104.195397


In [10]:
# exporting a csv to combine with happiness indicators
lat_merged_df.to_csv("data/nuke_coordinates_csv", index=False)